In [252]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score


In [253]:
# Do some preprocessing here
df_train_raw = pd.read_csv('downloaded/train.csv')
def do_preprocessing(df):
    print(df['Sex'].unique())

    sex_mapping = {
        'female': 0,
        'male': 1,
    }
    df['Sex'] = df['Sex'].map(sex_mapping)
    print(df['Sex'].unique())
    return df

df_train = do_preprocessing(df_train_raw)

# skipping non numeric columns and columns that have nulls in some rows
# TODO need better solution for nulls and numeric columns
# Age has some null columns
# Fare has 1 null in testing data
# eventually should probably be using of these columns 
# 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked'
training_columns = ['Pclass', 'Sex', 'SibSp', 'Parch']

['male' 'female']
[1 0]


In [254]:


y_train = df_train['Survived']



#x_train = df_train[training_columns]

x_train, x_test, y_train, y_test = train_test_split(df_train[training_columns], 
    y_train, test_size=0.3, random_state=0)

# feature scaling not needed for a decision tree but may want to add in later
#x_train.describe()


In [255]:
# let's just explore null values in this cell
for i in (['downloaded/train.csv', 'downloaded/test.csv']):
    df_tmp = pd.read_csv(i)
    print('total rows:{}'.format(len(df_tmp)))
    print(df_tmp.isnull().sum())


total rows:891
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
total rows:418
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [256]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)
tree.fit(x_train.as_matrix(), y_train.as_matrix())
y_pred = tree.predict(x_test)
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
x_test.describe()

Accuracy: 0.78


,Pclass,Sex,SibSp,Parch
count,268.000000,268.000000,268.000000,268.000000
mean,2.358209,0.630597,0.503731,0.354478
std,0.824739,0.483546,0.950057,0.742817
min,1.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000
50%,3.000000,1.000000,0.000000,0.000000
75%,3.000000,1.000000,1.000000,0.000000
max,3.000000,1.000000,5.000000,5.000000


# Last step generate the file to send to kaggle

In [257]:
df_test_raw = pd.read_csv('downloaded/test.csv')
df_test = do_preprocessing(df_train_raw)

x_test = df_test[training_columns]
result = tree.predict(x_test)

# last step is output the results to a csv
df_for_file = df_test[['PassengerId',]]
df_result = pd.DataFrame(result)
df_result.columns = ['Survived']

# get id of false pandas warning
pd.options.mode.chained_assignment = None

df_for_file['Survived'] = df_result[['Survived',]]

df_for_file.to_csv("out.csv", header = ['PassengerId', 'Survived'], index=False)

ValueError: could not convert string to float: 'male'